In [2]:
import math
from datetime import datetime
import numpy as np
import pandas as pd
import seaborn as sns # for visualiation
import matplotlib.pyplot as plt # plotting
import matplotlib
matplotlib.style.use('ggplot')
%matplotlib inline

df = pd.read_csv('./facebook_with_reactions.csv')
outcomes = df[['Rating']]

## drop useless columns 
df = df.drop(columns=['Debate', 'status_link', 'permalink_url', 'Post URL', 
                      'status_message', 'link_name', 'share_count', 'Unnamed: 0', 'account_id',
                      'status_id', 'status_type', 'status_published', 'post_id', 'reaction_count', 'Date Published'])

## change Rating to numeric for classification
df.Rating = df.Rating.replace(['mostly false', 'no factual content', 'mixture of true and false', 'mostly true'], [0,0,0,1]).astype(int)

## remove spaces in columns
df.columns = [c.replace(' ', '_') for c in df.columns]

## make dummy columns (make categories numeric)
df = pd.get_dummies(df)

In [3]:
#df = df.drop(df.query('Rating == 1').sample(frac=.5).index)
df.groupby('Rating').count()
# number of records for each outcome

,comment_count,num_reactions,num_comments,num_shares,num_likes,num_loves,num_wows,num_hahas,num_sads,num_angrys,...,Page_CNN Politics,Page_Eagle Rising,Page_Occupy Democrats,Page_Politico,Page_Right Wing News,Page_The Other 98%,Post_Type_link,Post_Type_photo,Post_Type_text,Post_Type_video
Rating,,,,,,,,,,,,,,,,,,,,,
0,554,554,554,554,554,554,554,554,554,554,...,554,554,554,554,554,554,554,554,554,554
1,1612,1612,1612,1612,1612,1612,1612,1612,1612,1612,...,1612,1612,1612,1612,1612,1612,1612,1612,1612,1612


In [4]:
# Because the data is unbalanced we need to weight the outcomes differently.
# Below is the rough estimates if we were to make each outcome worth the same. 
# Here we say that `mostly true` is stays the same and all other outcomes are weighted
# to equal the same as `mostly true`

###### THESE CLASS WEIGHTS ARE SUBJECT TO CHANGE BASED ON MODEL PERFORMANCE ######

## mostly true = 1
## mix of T/F ~ 7
## mostly false ~ 21
## no facts ~ 6

class_weight = {0: 21.,
                1: 6.,
                2: 7.,
                3: 1.}

## insert above parameter into .fit() function of model as **class_weight**

In [5]:
import mord as m
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

tr_feat, test_feat, tr_out, test_out = train_test_split(
   df.drop(columns='Rating'),      # features
   df.Rating,    # outcome
   random_state=0,
   test_size=0.20, # percentage of data to use as the test set
   
)

ModuleNotFoundError: No module named 'mord'

In [6]:
clf = LogisticRegression(random_state=0, solver='sag',
                         multi_class='multinomial').fit(tr_feat, tr_out)

preds = clf.predict(test_feat)

clf.predict_proba(test_feat) 

clf.score(test_feat, test_out)

NameError: name 'LogisticRegression' is not defined

In [18]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import Normalizer
from sklearn.preprocessing import StandardScaler

from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.feature_selection import f_classif

from sklearn.neural_network import MLPRegressor


from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import make_pipeline 

In [19]:
### Multi-layer Perceptron
mod = MLPRegressor(random_state=0)

scal = RobustScaler()
select_k_best = SelectKBest(f_classif)
m_pipe = make_pipeline(scal, select_k_best, mod)

m_param_grid = {
                'mlpregressor__activation':['relu', 'logistic'], 
                'mlpregressor__learning_rate':['adaptive'],
                'mlpregressor__solver':['lbfgs', 'adam'],
                'mlpregressor__hidden_layer_sizes':[(33,33,33), (500)]}
#mod.early_stopping=True # terminate training when validation score is not improving
mod.shuffle=True # shuffle samples when training

mlp = GridSearchCV(m_pipe, m_param_grid, cv=3, n_jobs=-1)
mlp.fit(tr_feat, tr_out);

c:\python\python37-32\lib\site-packages\sklearn\model_selection\_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


In [73]:
mlp.score(test_feat, test_out)

0.34078457159389053

In [72]:
mlp.best_params_

{'mlpregressor__activation': 'logistic',
 'mlpregressor__hidden_layer_sizes': 500,
 'mlpregressor__learning_rate': 'adaptive',
 'mlpregressor__solver': 'adam'}

In [13]:
df.describe()

,post_id,Rating,reaction_count,comment_count,num_reactions,num_comments,num_shares,num_likes,num_loves,num_wows,...,Date_Published_2016-09-20,Date_Published_2016-09-21,Date_Published_2016-09-22,Date_Published_2016-09-23,Date_Published_2016-09-26,Date_Published_2016-09-27,Post_Type_link,Post_Type_photo,Post_Type_text,Post_Type_video
count,2.166000e+03,2166.000000,2166.000000,2166.000000,2166.000000,2166.000000,2166.000000,2166.000000,2166.000000,2166.000000,...,2166.000000,2166.000000,2166.000000,2166.000000,2166.000000,2166.000000,2166.000000,2166.000000,2166.000000,2166.000000
mean,3.409298e+15,2.553555,5170.986611,441.455217,5296.478763,452.082179,3834.459834,4067.789935,262.443675,97.003232,...,0.138966,0.132041,0.128347,0.128809,0.179132,0.160203,0.770545,0.095106,0.001385,0.132964
std,3.879008e+15,0.837153,16866.706791,1324.224921,18159.453476,1510.787707,22345.925304,14940.861709,1397.449677,365.424746,...,0.345991,0.338613,0.334553,0.335066,0.383551,0.366879,0.420580,0.293429,0.037199,0.339614
min,5.510967e+14,0.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,1.247617e+15,2.000000,138.000000,34.000000,115.000000,29.000000,18.000000,78.000000,2.000000,2.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000
50%,1.291615e+15,3.000000,506.500000,126.500000,480.000000,118.500000,76.000000,297.000000,13.000000,7.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000
75%,1.541418e+15,3.000000,2249.750000,386.000000,2211.500000,374.000000,556.000000,1466.250000,66.000000,42.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000
max,1.015386e+16,3.000000,293333.000000,32419.000000,330144.000000,35799.000000,401465.000000,280694.000000,37168.000000,10326.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


# mord

In [22]:
import mord as m
from sklearn import linear_model, metrics, preprocessing

In [14]:
X=df.drop(columns=['Rating'])
y=df.Rating

In [15]:
tr_feat, test_feat, tr_out, test_out = train_test_split(
   X,      # features
   y,    # outcome
   random_state=0,
   test_size=0.20, # percentage of data to use as the test set
)

In [23]:
#dir(mord) doesn't have multiclassLogistic??
#clf2=m.MulticlassLogistic(alpha=1.0, verbose=0, maxiter=10000)

clf2 = m.LogisticAT(alpha=1.)
clf2.fit(tr_feat, tr_out)
print('Mean Absolute Error of LogisticAT %s' %
      metrics.mean_absolute_error(clf2.predict(tr_feat), tr_out))

Mean Absolute Error of LogisticAT 2.564665127020785


# Back Elimination

In [1]:
X=tr_feat
y=tr_out

from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression

logreg = LogisticRegression(solver='lbfgs', max_iter=1500)
rfe = RFE(logreg, 20)
rfe = rfe.fit(X, y.values.ravel())
print(rfe.support_)
#print(rfe.ranking_)

NameError: name 'tr_feat' is not defined

In [26]:
rfe.score(test_feat, test_out)

0.783410138248848